In [ ]:
import networkx as nx
import heapq

In [ ]:
G = nx.DiGraph()
edges = [(1, 2, 10),
         (2, 3, 1),
         (3, 4, 10),
         (1, 9, 20),
         (9, 2, 1),
         (2, 6, 1),
         (6, 8, 1),
         (8, 9, 1),
         (2, 8, 3),
         (8, 5, 15),
         (3, 5, 18),
         (5, 4, 1)]

# A 1
# B 2
# C 3
# D 4
# E 5
# F 6
# G 7
# H 8
# I 9

G.add_weighted_edges_from(edges)

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
def reverse(graph):
  Gr = nx.DiGraph()
  # for node in graph.nodes():
  #   Gr.add_node(node)

  Gr.add_edges_from((v,u,d) for u,v,d in graph.edges(data=True))
  return Gr

In [ ]:
GR = reverse(G)

In [ ]:
nx.draw(GR, with_labels=True)

In [ ]:
class Path:
  def __init__(self):
    self.route = {}
    self.length = 0
    self.LB1 = 0
    self.LB2 = 0
    self.lb = 0
    self.cls = None
    self.edges = {}


In [ ]:
distances = {}
isSettled = {}
PQ = []
parent = {}

for node in GR.nodes():
  distances[node] = float('inf')
  isSettled[node] = False
  parent[node] = None

dest = 4 # D vertexi destination noktamız
heapq.heappush(PQ, (0, dest))
distances[dest] = 0

In [ ]:
def ConstructPartialSPT(graph, v):
  global distances, isSettled, PQ, parent

  if isSettled[v]:
    return distances[v]

  while PQ:
    cost, node = heapq.heappop(PQ)

    if cost > distances[node]:
      continue

    if not isSettled[node]:
      isSettled[node] = True

      for neighbor, data in graph[node].items():

        if not isSettled[neighbor]:        
          new_cost = cost + data['weight']

          if new_cost < distances[neighbor]:
            distances[neighbor] = new_cost
            parent[neighbor] = node
            heapq.heappush(PQ, (new_cost, neighbor))

      if node == v:
        return distances[v]      
      
  return float('inf')

In [ ]:
ConstructPartialSPT(GR, 5)
print(distances)
print(isSettled)
print(parent)

In [ ]:
ConstructPartialSPT(GR, 2)
print(distances)
print(isSettled)
print(parent)

In [ ]:
ConstructPartialSPT(GR, 1)
print(distances)
print(isSettled)
print(parent)

In [ ]:
def LB1(length, node):
  return length + distances[node] 


In [ ]:
p = Path()
p.edges = {(1,2): 10,
           (2,3): 1,
           (3, 4): 10}
result = []
result.append(p)

In [ ]:
# path dict olarak tutuluyor (edgelerin listesi)
# result dict arrayi olarak tutuluyor
def LB2(new_path : Path, threshold):
  global result
  lb2 = 0
  new_path_edges = new_path.edges

  for old_path in result: 

    old_path_edges = old_path.edges

    common_edges = set(old_path_edges.keys()).intersection(set(new_path_edges.keys()))

    intersection_length = sum(old_path_edges[e] for e in common_edges)
    old_path_length = sum(e for e in old_path_edges.values())

    current_lb2 = intersection_length * (1+1/threshold) - old_path_length
    lb2 = max(lb2, current_lb2)

  return lb2


In [ ]:
path = Path()
path.edges = {(1,2): 10,
        (2,6): 1,
        (6, 7): 1,
        (7, 5): 15,
        (5, 4): 1}

LB2(path, 0.5)

In [ ]:
def lb(LB1, LB2):
  return max(LB1, LB2)

In [ ]:
def Sim(new_path, threshold):
  flag = True
  new_path_edges = new_path.edges

  for old_path in result: 
    old_path_edges = old_path.edges

    common_edges = set(old_path_edges.keys()).intersection(set(new_path_edges.keys()))
    intersection_length = sum(old_path_edges[e] for e in common_edges)

    union_edges = set(new_path_edges.keys()).union(set(old_path_edges.keys()))
    union_length = sum(new_path_edges[e] for e in union_edges if e in new_path_edges)
    union_length += sum(old_path_edges[e] for e in union_edges if e in old_path_edges)
    union_length -= intersection_length

    similarity = intersection_length / union_length

    if similarity >= threshold:
      flag = False
      break  

  return flag

In [ ]:
Sim(path, 0.5)

In [ ]:
# path dönen dijkstra
def dijkstra3(graph, src, dest):

  if src == dest:
    return {}, 0

  heap = [(0, src, [])]
  visited = set()

  while heap:
    cost, node, path = heapq.heappop(heap)

    if node in visited:
      continue
    visited.add(node)

    if node == dest:
      shortest_path = Path()

      for i in range(len(path)):
        if i < len(path) - 1:
          u, v = path[i], path[i+1]
          shortest_path.edges[(u,v)] = graph[u][v]['weight']
        else:
          u, v = path[i], dest
          shortest_path.edges[(u,v)] = graph[u][v]['weight']

      return shortest_path

    for neighbor, data, in graph[node].items():
      if neighbor not in visited:
        new_cost = cost + data['weight']
        heapq.heappush(heap, (new_cost, neighbor, path + [node]))

  return None, float('inf')

In [ ]:
dijkstra3(G, 1, 4).__dict__